In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tables = pd.read_html('https://www.prisonpolicy.org/blog/2020/12/02/jail-and-prison-covid-populations/')
print("{} Tables Found.".format(len(tables)))

2 Tables Found.


## Table 1

In [3]:
tables[0].head()

,County,State,March population,July population,Most recent population,Percent change from March to July,Percent change from July to the most recent date,Net percent change since March,March date,July date,Most recent date
0,Autauga,Ala.,171,158,193,-8%,20%,13%,3/10,7/1,11/15
1,Blount,Ala.,125,117,159,-6%,36%,27%,3/10,7/1,11/15
2,Chambers,Ala.,134,70,2,-48%,-97%,-99%,3/10,7/1,11/15
3,Cherokee,Ala.,110,73,76,-34%,4%,-31%,3/10,7/1,11/15
4,Clay,Ala.,38,31,31,-18%,0%,-18%,3/10,7/1,11/15


## Table 2

In [4]:
tables[1].head()

,State,January,January.1,May,May.1,July,July.1,August,August.1,September,September.1,October,October.1,November,November.1,Most recent,Most recent.1,Population Data Source
0,NaN,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,NaN
1,Arizona,42441,1/1,41386,5/1,40102,7/1,39125,8/21,38865,9/27,38741,10/7,38562,11/1,38385,11/19,"Vera’s People in Prison, 2019; ADCRR COVID-19 ..."
2,California,125365,1/15,119183,5/6,115201,7/1,104544,8/19,100747,9/30,101003,10/7,101658,11/4,100153,11/18,CDCR 2020 Weekly Total Population Reports
3,Connecticut,12284,1/1,10973,5/1,9945,7/1,9558,8/24,9391,9/30,9344,10/8,9350,11/1,9299,11/19,Department of Correction’s Total Population Co...
4,Georgia,53924,1/3,51294,5/1,49959,7/3,48274,8/21,46814,9/25,47368,10/2,46649,10/30,45893,11/13,GDC Friday Report


## Working From NYU's Raw Data

In [9]:
# response = requests.get('https://raw.githubusercontent.com/publicsafetylab/public-psl-jdi-pops/master/data.csv')
df = pd.read_csv('https://raw.githubusercontent.com/publicsafetylab/public-psl-jdi-pops/master/data.csv')
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier
0,Autauga,182,2020-01-01,AL,AL-Autauga
1,Autauga,184,2020-01-02,AL,AL-Autauga
2,Autauga,178,2020-01-03,AL,AL-Autauga
3,Autauga,180,2020-01-04,AL,AL-Autauga
4,Autauga,184,2020-01-05,AL,AL-Autauga


## Weekly Rolling Averages

In [11]:
grouped = df.groupby(['Facility_Identifier'])
for i, (name, g) in enumerate(grouped):
        temp = g['Population'].rolling(7).mean()
        for idx in temp.index:
            df.at[idx, 'RollingAvg'] = temp.loc[idx]
df.head(10)

,County,Population,Scrape_Date,State,Facility_Identifier,RollingAvg
0,Autauga,182,2020-01-01,AL,AL-Autauga,NaN
1,Autauga,184,2020-01-02,AL,AL-Autauga,NaN
2,Autauga,178,2020-01-03,AL,AL-Autauga,NaN
3,Autauga,180,2020-01-04,AL,AL-Autauga,NaN
4,Autauga,184,2020-01-05,AL,AL-Autauga,NaN
5,Autauga,183,2020-01-06,AL,AL-Autauga,NaN
6,Autauga,180,2020-01-07,AL,AL-Autauga,181.571429
7,Autauga,180,2020-01-08,AL,AL-Autauga,181.285714
8,Autauga,179,2020-01-09,AL,AL-Autauga,180.571429
9,Autauga,183,2020-01-10,AL,AL-Autauga,181.285714


In [12]:
df.tail()

,County,Population,Scrape_Date,State,Facility_Identifier,RollingAvg
236629,Teton,11,2020-11-28,WY,WY-Teton,11.571429
236630,Teton,12,2020-11-29,WY,WY-Teton,11.285714
236631,Teton,12,2020-11-30,WY,WY-Teton,11.285714
236632,Teton,12,2020-12-01,WY,WY-Teton,11.285714
236633,Teton,13,2020-12-02,WY,WY-Teton,11.714286


## Save Transformed Output to CSV

In [13]:
import datetime
today = datetime.datetime.now().strftime('%m-%d-%Y') #Create string of today's date
print(today)

12-03-2020


In [14]:
df.to_csv('PrisonWeeklyPopulations_as_of_{}.csv'.format(today)) #Save to file